In [1]:
%pip install pandas openpyxl transformers torch
%pip install llama-cpp-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_cpp import Llama
from concurrent.futures import ThreadPoolExecutor

model_path = "/Users/solar/OneDrive/Documents/Capstone/Capstone-Jupyter/mistral-7b-instruct-v0.1.Q4_K_M.gguf"
max_context = 4096
llm = Llama(
    model_path,
    n_ctx=max_context
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/solar/OneDrive/Documents/Capstone/Capstone-Jupyter/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 12

In [3]:
import json
import re

with open("metadata.json", "r", encoding="utf-8") as f:
    raw_metadata = json.load(f)

# Create a lookup table
metadata_lookup = {
    entry["code"]: f'{entry["name"]}. {entry["description"]}' 
    for entry in raw_metadata
}

def extract_codes(query):
    return re.findall(r"\[([A-Za-z0-9_]+)\]", query)

list_keywords = {"IN", "ANY", "NONE"}
null_keywords = {":NC": "Null type \"Not collected\"", ":NA": "Null type \"Not applicable\"", ":ND": "Null type \"Not disclosed\"", ":NI": "Null type \"No information\"", ":NM": "Null type \"Not meaningful\""}
def extract_nulls_and_lists(query):
    nulls = []
    lists = []
    contains_null = re.findall(r"\[([A-Za-z0-9_]+)\]\s+IS\s+(:[A-Z]{2})", query)
    for code, null_keyword in contains_null:
        nulls.append((code, null_keyword))
    
    contains_list = re.findall(r"\[([A-Za-z0-9_]+)\]\s+(IN|ANY|NONE)\b", query, re.IGNORECASE)
    for code, list_keyword in contains_list:
        lists.append((code, list_keyword.upper()))
    
    return nulls, lists

# Find the entry with the longest name + description combo
longest_entry = max(metadata_lookup.items(), key=lambda item: len(item[1]))

# Print the result
print(f'Longest entry code: {longest_entry[0]}')
print(f'Length: {len(longest_entry[1])}')
print(f'Content: {longest_entry[1]}')

Longest entry code: EUTaxManIntSerElcRevOverAlign
Length: 1353
Content: EU Taxon - Man, Instal, Serv of Elec Equip Overall Align. EU Taxon - Man, Instal, Serv of Elec Equip Overall Align: This factor identifies the overall alignment for Manufacture, installation, and servicing of high, medium and low voltage electrical equipment for electrical transmission and distribution that result in or enable a substantial contribution to climate change mitigation, covering the substantial contribution criteria, the do no significant harm criteria, and the minimum social safeguards. This is the aggregated result across all Taxonomy objectives. The possible values are: Aligned,Aligned (>90%),Aligned (>80%),Aligned (>70%),Aligned (>60%),Aligned (>50%),Aligned (>40%),Aligned (>30%),Aligned (>20%),Aligned (>10%),Aligned (>0%),Likely aligned (100%),Likely aligned (>90%),Likely aligned (>80%),Likely aligned (>70%),Likely aligned (>60%),Likely aligned (>50%),Likely aligned (>40%),Likely aligned (>30%),Li

In [4]:
def max_tokens(text, max_tokens=40):
    #moved this to use in below function
    tokens = llm.tokenize(text.encode("utf-8"))
    if len(tokens) <= max_tokens:
        return text
    truncated = llm.detokenize(tokens[:max_tokens]).decode("utf-8", errors="ignore")
    return truncated + "..."

def build_ordered_context(query, token_budget):
    #builds context in-order: null type definitions, enumerations for list types, and metadata lookups
    context_lines = []
    seen = set()
    used_tokens = 0

    null_hits, list_hits = extract_nulls_and_lists(query)
    codes_in_order = re.findall(r"\[([A-Za-z0-9_]+)\]", query)

    #tokenizer = llm.tokenizer
    for code in codes_in_order:
        if code in seen:
            continue
        seen.add(code)

        null_entry = next((kw for c, kw in null_hits if c == code), None)
        if null_entry:
            null_def = null_keywords.get(null_entry, f"No definition for {null_entry}")
            line = f"{code} = {null_entry} → {null_def}"
            tokens = len(llm.tokenize(line.encode("utf-8")))
            if used_tokens + tokens > token_budget:
                break
            context_lines.append(line)
            used_tokens += tokens

        base_meta = metadata_lookup.get(code, "No metadata found.")
        metadata_line = f"{code}: {max_tokens(base_meta, 100)}" 
        tokens = len(llm.tokenize(metadata_line.encode("utf-8")))
        if used_tokens + tokens > token_budget:
            break
        context_lines.append(metadata_line)
        used_tokens += tokens

        if any(c == code for c, _ in list_hits):
            enum_line = f"{code} (enumeration): {max_tokens(base_meta, 100)}"
            tokens = len(llm.tokenize(enum_line.encode("utf-8")))
            if used_tokens + tokens > token_budget:
                break
            context_lines.append(enum_line)
            used_tokens += tokens

    return "\n".join(context_lines)

def max_afforded_tokens(codes):
    return max(4096 // max(1, len(codes)), 100)


def translate_query(query, max_total_tokens=2048, max_output_tokens=256):
    codes = extract_codes(query)

    m_tokens = max_afforded_tokens(codes)

    # Initial prompt pieces
    instruction = "### Instruction:\nRephrase the following ESGish query into a concise natural English sentence. Each query is asking for all companies or issuers that match some paramater. Use the following metadata definitions for clarity:\n\n"
    query_part = f"\n\nQuery: {query}\n\n### Response:"

    # Tokenize instruction and query to calculate remaining token budget
    #tokenizer = llm.tokenize  # Built-in tokenizer
    instruction_tokens = len(llm.tokenize(instruction.encode("utf-8")))
    query_tokens = len(llm.tokenize(query_part.encode("utf-8")))
    token_budget = max_total_tokens - max_output_tokens - instruction_tokens - query_tokens

    # Build full context blocks for each code
    context = build_ordered_context(query, token_budget)
    print("query: ", query)
    print(context)

    """
    # Now iteratively add context blocks until budget is exhausted
    context = ""
    used_tokens = 0
    for block in context_blocks:
        block_tokens = len(tokenizer(block.encode("utf-8"))) + 1  # +1 for newline
        if used_tokens + block_tokens <= token_budget:
            context += block + "\n"
            used_tokens += block_tokens
        else:
            break  # stop once we're out of budget
    """
    # Final prompt
    prompt = instruction + context + query_part

    # Call model
    response = llm(prompt, max_tokens=max_output_tokens, temperature=0.1)
    return response["choices"][0]["text"].strip()




In [ ]:
import pandas as pd
from functools import partial
import time

#Reads and stores the Esgish queries
df = pd.read_excel("NullType_10Queries.xlsx")
queries = df["Esgish"].tolist()

#Ensures no overload and efficiency
batch_size = 100 
output_file = "translated_queries_TEST.xlsx"

#Looks at each query in each batch, calls the translate_query function, and stores it
for i in range(0, len(queries), batch_size):
    batch = queries[i:i + batch_size]
    translated_batch = []
    
    for query in batch:
        translated = translate_query(query)
        translated_batch.append(translated)
    
    df_batch = pd.DataFrame({
        "Esgish": batch,
        "English": translated_batch
    })

    #Makes a new file if needed, or adds onto the current file during each batch in case the program crashes at some point
    if i == 0:
        df_batch.to_excel(output_file, index=False)  
    else:
        with pd.ExcelWriter(output_file, mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
            df_batch.to_excel(writer, index=False, header=False, startrow=i + 1)
    
    print(f"Saved batch {i // batch_size + 1} to {output_file}")

query:  AND([ClawBackProvisionDisclosure] IS :ND,[CEOStockOwnershipGuideRatio] > '1')
ClawBackProvisionDisclosure = :ND → Null type "Not disclosed"
ClawBackProvisionDisclosure: Q155: Clawback Provision. Does the company have a clawback or malus provision? (Q155)
CEOStockOwnershipGuideRatio: Q145: CEO Stock Ownership Guidelines (% of Salary). What salary multiple of the CEO is subject to stock ownership guidelines? (Q145)


llama_perf_context_print:        load time =    4865.62 ms
llama_perf_context_print: prompt eval time =    4865.43 ms /   191 tokens (   25.47 ms per token,    39.26 tokens per second)
llama_perf_context_print:        eval time =    5296.44 ms /    33 runs   (  160.50 ms per token,     6.23 tokens per second)
llama_perf_context_print:       total time =   10167.99 ms /   224 tokens
Llama.generate: 190 prefix-match hit, remaining 1 prompt tokens to eval


query:  AND([ClawBackProvisionDisclosure] IS :ND,[CEOStockOwnershipGuideRatio] > '1')
ClawBackProvisionDisclosure = :ND → Null type "Not disclosed"
ClawBackProvisionDisclosure: Q155: Clawback Provision. Does the company have a clawback or malus provision? (Q155)
CEOStockOwnershipGuideRatio: Q145: CEO Stock Ownership Guidelines (% of Salary). What salary multiple of the CEO is subject to stock ownership guidelines? (Q145)


llama_perf_context_print:        load time =    4865.62 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    5488.63 ms /    34 runs   (  161.43 ms per token,     6.19 tokens per second)
llama_perf_context_print:       total time =    5494.40 ms /    35 tokens
Llama.generate: 51 prefix-match hit, remaining 169 prompt tokens to eval


query:  AND([ClimateEmissionsFiscalYear] == '2022',[ClimateParentEntityID] IS :NA)
ClimateEmissionsFiscalYear: GHG Emissions - Fiscal Year. GHG Emissions - Fiscal Year: This factor provides the fiscal year associated with the corresponding GHG emissions values.
ClimateParentEntityID = :NA → Null type "Not applicable"
ClimateParentEntityID: Climate Parent Entity ID. Climate Parent Entity ID: This factor provides the identifier of the issuer‘s parent, from which the climate data is assigned. The climate data from the parent company is assigned to a subsidiary based on criteria such as industry classification, ownership structure, and business activities.


...

query:  OR(AND([PornographyNotAccessibeToMinors] IS :NA,[PornographyDistMinRev] IS :NA),AND([PornographyDistMinRev] <= '0.02',[PornographyNotAccessibeToMinors] False))
PornographyNotAccessibeToMinors = :NA → Null type "Not applicable"
PornographyNotAccessibeToMinors: Pornography - Not accessible to minors. Pornography - Not accessible to minors: This factor identifies pornographic materials which are specifically marked as not accessible to minors or are prohibited/restricted for those persons below 18 years of age.
PornographyDistMinRev = :NA → Null type "Not applicable"
PornographyDistMinRev: Pornography - Distribution Minimum Percentage of Revenues (%). Pornography - Distribution Minimum Percentage of Revenues (%): This factor identifies the minimum share of revenue estimated or reported to be derived from involvement in the distribution of pornography as a percentage of the issuers’ total revenue in the latest financial year.


llama_perf_context_print:        load time =    4865.62 ms
llama_perf_context_print: prompt eval time =    6071.49 ms /   251 tokens (   24.19 ms per token,    41.34 tokens per second)
llama_perf_context_print:        eval time =   15037.55 ms /    95 runs   (  158.29 ms per token,     6.32 tokens per second)
llama_perf_context_print:       total time =   21128.50 ms /   346 tokens
Llama.generate: 50 prefix-match hit, remaining 273 prompt tokens to eval


query:  OR([TobaccoInvolvement] ANY 'Distribution|Services',[TobaccoInvolvement] IS :NC)
TobaccoInvolvement = :NC → Null type "Not collected"
TobaccoInvolvement: Tobacco - Involvement Tie. Tobacco - Involvement Tie: This factor identifies issuers engaged in the production, distribution, or provision of services related to tobacco. The "Production" value identifies issuers engaged in manufacturing and producing tobacco products, as well as companies that grow or process raw tobacco leaves, "Distribution" includes issuers engaged in the wholesale or retail distribution of tobacco products, and "Services"  identifies iss...
TobaccoInvolvement (enumeration): Tobacco - Involvement Tie. Tobacco - Involvement Tie: This factor identifies issuers engaged in the production, distribution, or provision of services related to tobacco. The "Production" value identifies issuers engaged in manufacturing and producing tobacco products, as well as companies that grow or process raw tobacco leaves, "Dist

llama_perf_context_print:        load time =    4865.62 ms
llama_perf_context_print: prompt eval time =    6650.51 ms /   273 tokens (   24.36 ms per token,    41.05 tokens per second)
llama_perf_context_print:        eval time =    5861.69 ms /    37 runs   (  158.42 ms per token,     6.31 tokens per second)
llama_perf_context_print:       total time =   12518.54 ms /   310 tokens


Saved batch 1 to translated_queries_TEST.xlsx


In [ ]:
import pandas as pd

output_file = "translated_queries_TEST5.xlsx"

# Extract the English column
df = pd.read_excel(output_file)
english_queries = df["English"].tolist()

# Function to request a comprehensibility rating from the model
def rate_comprehensibility(text, max_tokens=256):
    # Adjust the prompt to ask the model for a rating from 1 to 10
    prompt = f"""### Instruction:
Please rate the following text's comprehensibility from 1 to 10, where:
- 1 = Completely incomprehensible, nonsensical, or full of errors.
- 5 = Understandable with effort; some awkwardness, complexity, or minor errors.
- 10 = Perfectly clear, natural, and easy to understand.

Here are some examples:
Text: "asjdk asjd aksd" → Rating: 1
Text: "Provide list companies ESG data incomplete understandable" → Rating: 4
Text: "Please provide a list of companies with complete ESG data." → Rating: 9

Now, rate this text:
Text: {text}

### Response:"""
    # Call model (adjust temperature and other params as needed)
    response = llm(prompt, max_tokens=max_tokens, temperature=0.2)
    rating = response["choices"][0]["text"].strip()
    print(rating)
    # Ensure the response is a valid number between 1 and 10
    try:
        rating = int(rating)
        if 1 <= rating <= 10:
            return rating
    except ValueError:
        pass
    return None  # Return None if no valid rating is obtained

# List to store ratings
ratings = []

# Iterate through each English translation and get a rating
for query in english_queries:
    rating = rate_comprehensibility(query)
    ratings.append(rating)

# Add the ratings as a new column to the dataframe
df["Comprehensibility Rating"] = ratings

# Save the updated dataframe with ratings to a new Excel file
df.to_excel(output_file, index=False)

print("Comprehensibility ratings added and saved to 'translated_queries_with_ratings.xlsx'")


Llama.generate: 63 prefix-match hit, remaining 48 prompt tokens to eval
llama_perf_context_print:        load time =    4865.62 ms
llama_perf_context_print: prompt eval time =    1261.82 ms /    48 tokens (   26.29 ms per token,    38.04 tokens per second)
llama_perf_context_print:        eval time =     316.26 ms /     2 runs   (  158.13 ms per token,     6.32 tokens per second)
llama_perf_context_print:       total time =    1579.26 ms /    50 tokens
Llama.generate: 90 prefix-match hit, remaining 20 prompt tokens to eval


8


llama_perf_context_print:        load time =    4865.62 ms
llama_perf_context_print: prompt eval time =     556.81 ms /    20 tokens (   27.84 ms per token,    35.92 tokens per second)
llama_perf_context_print:        eval time =     391.39 ms /     2 runs   (  195.70 ms per token,     5.11 tokens per second)
llama_perf_context_print:       total time =     948.92 ms /    22 tokens
Llama.generate: 65 prefix-match hit, remaining 44 prompt tokens to eval


8


llama_perf_context_print:        load time =    4865.62 ms
llama_perf_context_print: prompt eval time =    1127.64 ms /    44 tokens (   25.63 ms per token,    39.02 tokens per second)
llama_perf_context_print:        eval time =     314.37 ms /     2 runs   (  157.18 ms per token,     6.36 tokens per second)
llama_perf_context_print:       total time =    1442.72 ms /    46 tokens
Llama.generate: 65 prefix-match hit, remaining 21 prompt tokens to eval


...

7


llama_perf_context_print:        load time =    4865.62 ms
llama_perf_context_print: prompt eval time =    1320.97 ms /    51 tokens (   25.90 ms per token,    38.61 tokens per second)
llama_perf_context_print:        eval time =     345.12 ms /     2 runs   (  172.56 ms per token,     5.80 tokens per second)
llama_perf_context_print:       total time =    1666.94 ms /    53 tokens
Llama.generate: 63 prefix-match hit, remaining 26 prompt tokens to eval


8


llama_perf_context_print:        load time =    4865.62 ms
llama_perf_context_print: prompt eval time =     736.69 ms /    26 tokens (   28.33 ms per token,    35.29 tokens per second)
llama_perf_context_print:        eval time =     323.55 ms /     2 runs   (  161.78 ms per token,     6.18 tokens per second)
llama_perf_context_print:       total time =    1061.14 ms /    28 tokens


8
Comprehensibility ratings added and saved to 'translated_queries_with_ratings.xlsx'
